In [ ]:
from pfs.lam.detAnalysis import *
from pfs.lam.imageAnalysis import *

from pfs.lam.fileHandling import *
import glob

In [ ]:
#%matplotlib widget

import matplotlib.pyplot as plt
#plt.style.use('dark_background')
from matplotlib import style
style.use('ggplot')

In [ ]:
import lsst.daf.persistence as dafPersist

In [ ]:
def plotOnePeak(image, cx,cy, roi_size=30, doBck=False, nRows=5, vmin=None, vmax=None, verbose=False):
    indx = cy
    indy = cx

    if type(image) is str:
        hdulist = fits.open(image, "readonly")
        image = hdulist[1].data
    

    data = np.copy(image)
    outer_data, inner_data = getRois(data, cx, cy, inner_size=5, outer_size=roi_size, doBck=doBck, nRows=nRows)    
    m, s = np.mean(outer_data), np.std(outer_data)
    if verbose:
        print(f"mean: {m}")
        print(f"std: {s}")
    vmin = vmin if vmin is not None else m-s
    vmax = vmax if vmax is not None else m+s
    
    fig, (ax, ax2) = plt.subplots(ncols=2, constrained_layout=True)
    #ax = plt.subplot(111)
    im = ax.imshow(outer_data,interpolation="none", origin="lower", vmin=vmin, vmax=vmax)
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    
    m = np.log10(np.where(outer_data>0, outer_data, -1))

    #ax2 = plt.subplot(121)
    im = ax2.imshow(m,interpolation="none", origin="lower",cmap="gray")
    
    plt.colorbar(im, cax=cax)
    plt.show()

In [ ]:
def plotRoiPeak(image, peak_list, roi_size=20, raw=False, scale=True, verbose=False, savePlotFile=False):
    if type(image) is str:     
        hdulist = fits.open(image, "readonly")
        image = hdulist[1].data
    
    plist = pd.read_csv(peak_list) if type(peak_list) is str else peak_list

#    plist = plist.sort_values(["X","Y"], ascending=[True,False])
    
    nbfiber = len(plist.fiber.unique())
    nbwave = len(plist.wavelength.unique())
    # have a list of fiber from 0 to nbfiber
    listwavestoplot = pd.DataFrame(plist.wavelength.unique(), columns=["wavelength"])

    listfiberstoplot = pd.DataFrame(plist.fiber.unique(), columns=["fiber"])
    
    print("#Fiber= %d and #wavelength= %d"%(nbfiber, nbwave))
    f, axarr = plt.subplots(nbwave, nbfiber,  sharex='col', sharey='row',figsize=(12,8))
#    print(axarr.shape)
    vmin=None
    vmax=None
    
    for (wave, fiber), group in plist.groupby(['wavelength','fiber']):
        k = listwavestoplot[listwavestoplot.wavelength == wave].index.tolist()[0]
        i = listfiberstoplot[listfiberstoplot.fiber == fiber].index.tolist()[0]
        if verbose:
            print(k,i)
            print(f"px {group['px']}    py: {group['py']}")
        #cut_data = image[int(indx-roi_size/2):int(indx+roi_size/2), int(indy-roi_size/2):int(indy+roi_size/2)]
        cut_data = selectRoi(image, group["px"], group["py"], roi_size=roi_size)
        if nbwave == 1 and nbfiber == 1:
            axarr.set_title(f"{str(fiber)}, {str(wave)}")
            axarr.imshow(cut_data,interpolation="none", origin="lower", vmin=vmin, vmax=vmax)
        else:
            #axarr[nbwave -1 -k, nbfiber - i -1].set_title(f"{fiber}, {wave:.2f}")
            #axarr[nbwave -1 -k, nbfiber - i -1].label_outer()
            axarr[nbwave -1 -k, nbfiber - i -1].imshow(cut_data,interpolation="none", origin="lower", vmin=vmin, vmax=vmax)
#            axarr[nbwave -1 -k, nbfiber - i -1].set_ylabel(f"{wave:.2f}")
#            axarr[nbwave -1 -k, nbfiber - i -1].set_xlabel(fiber)
            axarr[nbwave -1 -k, nbfiber - i -1].grid(False)

                
    f.subplots_adjust(hspace=0.5,wspace=0.5)

    for ax, wave in zip(axarr[:,0], listwavestoplot.sort_index(ascending=False).wavelength.values) :
            ax.set_ylabel(f"{wave:.2f}", rotation='horizontal', ha='right', fontsize=20)
#            ax.set_xlabel(ax.get_xlabel(), rotation='vertical', fontsize=20)
            ax.set_yticklabels('')
            ax.set_xticklabels('')
            ax.set_frame_on(False)
    for ax, fiber in zip(axarr[-1,:], listfiberstoplot.sort_index(ascending=False).fiber.values):
#            ax.set_ylabel(ax.get_ylabel(), rotation='horizontal', ha='right', fontsize=20)
            ax.set_xlabel(fiber, rotation='vertical', fontsize=20)
            ax.set_yticklabels('')
            ax.set_xticklabels('')
            ax.set_frame_on(False)

    plt.gcf().set_facecolor('w')
    if doSave:
        fig.patch.set_alpha(0.5)
        plt.savefig(savePlotFile+f"_roi_all.png")
                  
    plt.show()


In [ ]:
cluster = True

In [ ]:
if cluster :
    imgPath = '/net/SRVSTK20C/drp/fmadec/analysis/subaru/sm1/'
    dataPath = "/net/SRVSTK20C/drp/fmadec/analysis/subaru/sm1/"
    drpPath = "/net/SRVSTK20C/drp/cluster"
    repo = "repo"
else:
    imgPath = '/media/fmadec/PfsData/PFS/fmadec/analysis/subaru/sm1/'
    dataPath = imgPath
    drpPath = "/home/fmadec/pfs/subaruData/"    
    repo = "sm1-newfoc-20200721"
    repo = "sm1"

In [ ]:
verbose = True

In [ ]:
# 
ExpIds = [169]
rerun = "sm1-octob2020" #"sm1-march2020"
cam = "r1"
roi_size = 24
doBck = True

Sources = ["hgar"]

In [ ]:
#repo = "drp"
#drpPath = "/home/fmadec/pfs/subaruData/"
arm = cam[0]
sm = int(cam[1])
print(arm, sm)

In [ ]:
SM = "SM1"
site = "Subaru"

In [ ]:
#extra = "Imqual_LAM_peaklist_2020July"
extra = ""

In [ ]:
files = []
for ExpId in ExpIds :
    csvPath = imgPath+"Exp"+str(ExpId)+"/"+rerun+"_roi"+str(roi_size)+"/doBck"+str(doBck)+"/"+extra
    dataPath = csvPath
#    print(dataPath)

    searchFile = f"{csvPath}Imquality_{cam}_Exp{ExpId}*"
    print(searchFile)
    files.extend(glob.glob(f"{searchFile}"))

if verbose:
    print(*files, sep="\n")

In [ ]:
filelist = files

In [ ]:
comment = extra + f"doBck{doBck}"
savePlotFile = imgPath+f"{SM}_{site}_{cam}_ExpIds_{'-'.join(str(x) for x in ExpIds)}_{'-'.join(str(x) for x in Sources)}"
savePlotFile = savePlotFile+f"_{comment}"
print(savePlotFile)
savePlotScatterFile = imgPath+f"{SM}_{site}_{cam}_ExpIds_{'-'.join(str(x) for x in ExpIds)}_{'-'.join(str(x) for x in Sources)}_scatter"
savePlotScatterFile = savePlotScatterFile+f"_{comment}"
print(savePlotScatterFile)
title = f"{SM} {site} {cam} ExpIds {' '.join(str(x) for x in ExpIds)} {' '.join(str(x) for x in Sources)}"
title = title + f"\n{comment}"
print(title)

In [ ]:
ob = pd.concat([pd.read_csv(filen) for filen in filelist ], ignore_index=True)
#ob = pd.concat(map(pd.read_csv,files)).reset_index().sort_values(by="motor1")


In [ ]:
ob["sep_2ndM"]= ob.apply(lambda x: np.mean([x["sep_x2"],x["sep_y2"]]) , axis=1)

In [ ]:
ob.element.unique()

In [ ]:
ob.wavelength.unique()

In [ ]:
filelist[0].split('/')[-1]

# do some filtering if needed/wanted

In [ ]:
df = ob

In [ ]:
len(ob[(ob.sep_flag >1) ])

In [ ]:
df = ob[(ob.sep_flag <1) ]

In [ ]:
waves = ob.wavelength.unique()
print(waves)

In [ ]:
fibers = ob.fiber.unique()
print(fibers)

####  Filtering of 
#ExpIds = [2605, 2606] 
#rerun = "sm1-focus-sep2020"
#cam = "r1"
#roi_size = 24
#doBck = True

#Sources = ["argon", "neon"] 

#because of saturation or CR

ind_wave = 8
print(waves[ind_wave])
df = ob[ob.wavelength != waves[ind_wave]]

ind_wave = 11
print(waves[ind_wave])
df = df[df.wavelength != waves[ind_wave]]

df = df[(df.EE5>0.3) & (df.fwhm < 6) ] #=> checked CR 

In [ ]:
print(len(ob))
print(len(df))
print(len(ob)-len(df))
print(100*(len(ob)-len(df))/len(ob))

In [ ]:
df = df[(df.sep_flag <1) ]

In [ ]:
doSavePlot = True

In [ ]:
pdf = df[(df.EE5<0.860)]
print(len(pdf))

In [ ]:
pdf = df[(df.fwhm>3.5) | (df.fwhm<2.4)]
print(len(pdf))

In [ ]:
pdf.set_index(["wavelength", "fiber"])[["fwhm", "brightness", "EE5"]]

In [ ]:
ind = 0

In [ ]:
visitId = int(pdf.visit.iloc[ind])
print(visitId)
print(pdf.wavelength.iloc[ind])
print(pdf.fiber.iloc[ind])
print(pdf.EE5.iloc[ind])
#visitId = 3449 

In [ ]:
print(f"{drpPath}/{repo}/rerun/{rerun}/detrend/")

In [ ]:
butler = dafPersist.Butler(f"{drpPath}/{repo}/rerun/{rerun}/detrend/")
calExp = butler.get("calexp", visit=visitId, arm=arm)

In [ ]:
cx = pdf.px.iloc[ind]
cy = pdf.py.iloc[ind]
print(cx,cy)
#imagefile = imgPath+ pdf.file.iloc[ind]
#print(imagefile)

In [ ]:
plotRoiPeak(calExp.image.array, df[df.visit == visitId].reset_index(), roi_size=16, verbose=False)

In [ ]:
plotRoiPeak(calExp.image.array, df[df.visit == visitId].reset_index(), roi_size=16, verbose=False, scale=True)

In [ ]:
ob.wavelength.unique()

In [ ]:

plotOnePeak(calExp.image.array,cx, cy, roi_size=24, vmin=0, vmax= 10, doBck=True, verbose=True)

In [ ]:
getEE(calExp.image.array,cx, cy,doBck=True, roi_size=24)

In [ ]:

for index, row in pdf.iterrows():
    visitId = int(row.visit)
    calExp2 = butler.get("calexp", visit=visitId, arm=arm)
    plotOnePeak(calExp2.image.array,row.px, row.py, roi_size=24, vmin=0, vmax= 10, doBck=True, verbose=True) 